In [1]:
from meta_opt.utils.experiment_utils import make, process_results, bcolors, plot, get_final_cparams

import os
import matplotlib.pyplot as plt
from IPython.display import HTML
import itertools
import dill as pkl

# Load

In [34]:
# configuration
CFG = {
    # training options are None because we arent training :)
    'workload': None,
    'num_iters': None,
    'eval_every': None,
    'num_eval_iters': None,
    'batch_size': None,
    'full_batch': True,
    'reset_every': None,

    # experiment options
    'experiment_name': 'wmt_base_baselines',
    'load_checkpoint': True,
    'overwrite': False,  # whether to allow us to overwrite existing checkpoints or throw errors
    'directory': os.path.abspath("../"),
}
results = make(CFG)  # save to temp var `_results` so that we dont immediately overwrite `results` in case somethings wrong
processed_results = process_results(CFG, results)
# processed_results = pkl.load(open('{}/data/{}_processed.pkl'.format(CFG['directory'], CFG['experiment_name']), 'rb'))

using cpu for jax
results will be stored at: /Users/evandigiorno/Desktop/meta-opt/data/wmt_base_baselines_*.pkl
we will try to load experiment checkpoint first
	loaded checkpoint from /Users/evandigiorno/Desktop/meta-opt/data/wmt_base_baselines_raw.pkl, containing ['rsqrt', 'adamw']
note: using full_batch means we will never eval
[[ 202.34400338]
 [ 203.75807755]
 [ 203.82456081]
 ...
 [7541.41799413]
 [7541.48666127]
 [7541.54280887]]
[[9.756444 ]
 [9.746006 ]
 [9.759505 ]
 ...
 [4.5592437]
 [4.373619 ]
 [4.585631 ]]
[[0.00000000e+00]
 [1.49989501e-06]
 [3.00025567e-06]
 ...
 [1.20001798e-03]
 [1.20001205e-03]
 [1.20000599e-03]]
[[0.20762508]
 [0.2345091 ]
 [0.21651964]
 [0.18368505]
 [0.179319  ]
 [0.17424032]
 [0.18166953]
 [0.17664006]
 [0.17997931]
 [0.17903104]
 [0.18049175]
 [0.18122685]
 [0.17605995]
 [0.16257457]
 [0.16845565]
 [0.17618462]
 [0.16969834]
 [0.17013933]
 [0.17387593]
 [0.16139406]
 [0.17557102]
 [0.1769482 ]
 [0.17124484]
 [0.16985218]
 [0.17079663]
 [0.16490363

In [30]:
# from meta_opt.utils.experiment_utils import save_checkpoint
# ret = {}
# for k in results[0].keys():
#     ret[k] = [r[k][0] for r in results]
#     for i in range(len(results)):
#         ret[k][i]['args']['experiment_name'] = 'cifar_wd'
# save_checkpoint(CFG, ret)

In [31]:
# baseline_results = pkl.load(open('{}/data/{}_processed.pkl'.format(CFG['directory'], 'mnist_baselines'), 'rb'))
# r = {}
# for k in baseline_results.keys(): 
#     r[k] = baseline_results[k]
#     for _k, _v in processed_results[k].items():
#         r[k][_k] = _v
# processed_results = r

# Plot
Plot and also animate the values taken by the $\{M_h\}_{h=1}^H$ coefficients during training. Each $M_h$ multiplies a disturbance from $h$ training steps ago (i.e. 0 is most recent in this plot).

In [ ]:
# ----------------------------------------
# plot a particular set of experiments
# ----------------------------------------
keys_to_plot = {
    'sgd': 'sgd',
    'momentum': 'momentum',
    'adamw': 'adamw',
    'dadamw': 'd-adap',
    'mechadamw': 'mechanic',
    # 'rmsprop',
    'hgd': 'hgd',
    'cf': 'ours (finetuned)',
    # 'ncf': 'ours (finetuned)',
    'frozen': 'ours (frozen)',
}

plots_to_make = {
              'loss': 'Training Loss',
              'eval_loss': 'Evaluation Loss',
              'eval_acc': 'Evaluation Accuracy',
              'bleu': 'BLEU Score',
              'M': 'Learned Coefficients',
              'anim': 'Animation',
               'lr': 'Learning Rate',
}

# ----------------------------------------
# OR just plot em via regex
# ----------------------------------------
# keys_to_plot = '.*long.*'  # specific regex
keys_to_plot = '.*'  # anything

(fig, ax), anim = plot(results, processed_results, keys_to_plot, plots_to_make, anim_bounds=(-0.15, 0.05), smoothing=20)
ax[0].set_ylim(0.,10)
# ax[0].set_xlim(0, 4000)
# ax[1].set_ylim(0.6, 0.95)
# ax[2].set_ylim(0.95, 0.99)
# ax[3].set_ylim(0.95, 0.99)
# ax[4].set_ylim(-0.1, 40)
# ax[5].set_ylim(-0.05, 0.05)
# plt.savefig('{}/figs/{}.pdf'.format(CFG['directory'], CFG['experiment_name']))
plt.show()

if anim is not None:
    h = HTML(anim.to_html5_video())
    display(h)

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


ValueError: max() arg is an empty sequence